In [40]:
import os
from dataclasses import dataclass
import pandas as pd
import numpy as np
import torch
import torchaudio
import requests
from num2words import num2words

'forty-two'

In [11]:
df = pd.read_pickle("../data/text_audio_video_emotion_data.pkl")
df = df[df["size_padding"] < 1000000]
# df.iloc[0]

In [5]:
x = df.iloc[0]['text']
x.replace(", " , "")

'hey uh i m really really sorry about what happened in the cafeteria today'

In [12]:
def print_df(path , text):
    return path+text
    # print(f"path is {path} and text is {text}")

In [14]:
from tqdm import tqdm
tqdm.pandas()

df['lol'] = df.progress_apply(lambda x: print_df(x.audio_path , x.text) , axis = 1)


100%|██████████| 13506/13506 [00:00<00:00, 26436.65it/s]


In [37]:
df.iloc[424]['lol']


'../../data/train_splits_wav/dia44_utt0.wavand then , and then you said that thing about , about bringing the mesozoic era in the 21st century'

In [39]:
num2words(21 , ordinal=True)

'twenty-first'

In [35]:
t = "okay , you have 19 questions left use them wisely come on joey ! you can t win if you don t ask any"
t.split()

['okay',
 ',',
 'you',
 'have',
 '19',
 'questions',
 'left',
 'use',
 'them',
 'wisely',
 'come',
 'on',
 'joey',
 '!',
 'you',
 'can',
 't',
 'win',
 'if',
 'you',
 'don',
 't',
 'ask',
 'any']

In [ ]:
transc_list = t.split()
for i , word in enumerate(transc_list):
    if word.isnumeric():
        transc_list[i] = str(num2words(word)).replace("-" , " ")

In [34]:
' '.join(transc_list)


'okay you have nineteen questions left use them wisely come on joey you cant win if you dont ask any'

In [24]:
df.head(14)

,emotion,dialog,utterance,text,num_words,split,audio_path,name,label,num_channels,audio_shape,size_padding,video_path,lol
8535,3,216,0,"hey , uh , i m really , really sorry about wha...",17,train,../../data/train_splits_wav/dia216_utt0.wav,dia216_utt0,sadness,6,166571,999426,../../data/train_video/dia216_utt0.mp4,../../data/train_splits_wav/dia216_utt0.wavhey...
9128,3,19,18,"i mean , well , 'cause when i was growing up ,...",43,val,../../data/val_splits_wav/dia19_utt18.wav,dia19_utt18,sadness,6,166229,997374,../../data/val_video/dia19_utt18.mp4,../../data/val_splits_wav/dia19_utt18.wavi mea...
6246,0,243,3,"okay , very cute braces anyway y know what , the",11,test,../../data/test_splits_wav/dia243_utt3.wav,dia243_utt3,neutral,6,166229,997374,../../data/test_video/dia243_utt3.mp4,../../data/test_splits_wav/dia243_utt3.wavokay...
8074,2,12,6,"well , i i got this blinding pain in my stomac...",34,train,../../data/train_splits_wav/dia12_utt6.wav,dia12_utt6,fear,6,166229,997374,../../data/train_video/dia12_utt6.mp4,../../data/train_splits_wav/dia12_utt6.wavwell...
7650,1,4,11,"ross geller , why do i know that name ? it s u...",20,val,../../data/val_splits_wav/dia4_utt11.wav,dia4_utt11,surprise,6,166229,997374,../../data/val_video/dia4_utt11.mp4,../../data/val_splits_wav/dia4_utt11.wavross g...
9276,3,84,2,"well , i i got this blinding pain in my stomac...",34,test,../../data/test_splits_wav/dia84_utt2.wav,dia84_utt2,sadness,6,166229,997374,../../data/test_video/dia84_utt2.mp4,../../data/test_splits_wav/dia84_utt2.wavwell ...
1179,0,263,5,but no chandler is in an accident and can t pe...,31,train,../../data/train_splits_wav/dia263_utt5.wav,dia263_utt5,neutral,6,165547,993282,../../data/train_video/dia263_utt5.mp4,../../data/train_splits_wav/dia263_utt5.wavbut...
8469,3,84,6,"oh , wait , joshua ! joshua ! yeah , well , th...",16,train,../../data/train_splits_wav/dia84_utt6.wav,dia84_utt6,sadness,6,165547,993282,../../data/train_video/dia84_utt6.mp4,"../../data/train_splits_wav/dia84_utt6.wavoh ,..."
12995,6,864,4,i m monica s maid of honor okay ? don t try to...,17,train,../../data/train_splits_wav/dia864_utt4.wav,dia864_utt4,anger,6,165547,993282,../../data/train_video/dia864_utt4.mp4,../../data/train_splits_wav/dia864_utt4.wavi m...
1337,0,301,10,"listen honey , if i'm hogging the ball too muc...",28,train,../../data/train_splits_wav/dia301_utt10.wav,dia301_utt10,neutral,6,164864,989184,../../data/train_video/dia301_utt10.mp4,../../data/train_splits_wav/dia301_utt10.wavli...


In [18]:
@dataclass
class Point:
  token_index: int
  time_index: int
  score: float

# Merge the labels
@dataclass
class Segment:
  label: str
  start: int
  end: int
  score: float

  def __repr__(self):
    return f"{self.label}\t({self.score:4.2f}): [{self.start:5d}, {self.end:5d})"

  @property
  def length(self):
    return self.end - self.start

In [19]:
def get_trellis(emission, tokens, blank_id=0):
  num_frame = emission.size(0)
  num_tokens = len(tokens)

  # Trellis has extra diemsions for both time axis and tokens.
  # The extra dim for tokens represents <SoS> (start-of-sentence)
  # The extra dim for time axis is for simplification of the code.
  trellis = torch.full((num_frame+1, num_tokens+1), -float('inf'))
  trellis[:, 0] = 0
  for t in range(num_frame):
    trellis[t+1, 1:] = torch.maximum(
        # Score for staying at the same token
        trellis[t, 1:] + emission[t, blank_id],
        # Score for changing to the next token
        trellis[t, :-1] + emission[t, tokens],
    )
  return trellis

def backtrack(trellis, emission, tokens, blank_id=0):
  # Note:
  # j and t are indices for trellis, which has extra dimensions
  # for time and tokens at the beginning.
  # When refering to time frame index `T` in trellis,
  # the corresponding index in emission is `T-1`.
  # Similarly, when refering to token index `J` in trellis,
  # the corresponding index in transcript is `J-1`.
  j = trellis.size(1) - 1
  t_start = torch.argmax(trellis[:, j]).item()

  path = []
  for t in range(t_start, 0, -1):
    # 1. Figure out if the current position was stay or change
    # Note (again):
    # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
    # Score for token staying the same from time frame J-1 to T.
    stayed = trellis[t-1, j] + emission[t-1, blank_id]
    # Score for token changing from C-1 at T-1 to J at T.
    changed = trellis[t-1, j-1] + emission[t-1, tokens[j-1]]

    # 2. Store the path with frame-wise probability.
    prob = emission[t-1, tokens[j-1] if changed > stayed else 0].exp().item()
    # Return token index and time index in non-trellis coordinate.
    path.append(Point(j-1, t-1, prob))

    # 3. Update the token
    if changed > stayed:
      j -= 1
      if j == 0:
        break
  else:
    raise ValueError('Failed to align')
  return path[::-1]

def merge_repeats(path , ratio , transcript , sr ):
  i1, i2 = 0, 0
  segments = []
  while i1 < len(path):
    while i2 < len(path) and path[i1].token_index == path[i2].token_index:
      i2 += 1
    score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
    segments.append(Segment(transcript[path[i1].token_index], path[i1].time_index, path[i2-1].time_index + 1, score))
    i1 = i2
  return (segments[0].start * ratio)/sr , (segments[-1].end * ratio)/sr

def get_times(path , text , bundle , model , labels , sample_rate):
  with torch.inference_mode():
    waveform, _ = torchaudio.load(path)
    emissions, _ = model(waveform)
    emissions = torch.log_softmax(emissions, dim=-1)
  emission = emissions[0].cpu().detach()

  dictionary  = {c: i for i, c in enumerate(labels)}
  transcript = text.upper().replace(" " , "|")
  tokens = [dictionary[c] for c in transcript]


  trellis = get_trellis(emission, tokens)

  path = backtrack(trellis, emission, tokens)
  ratio = waveform[0].size(0) / (trellis.size(0) - 1)
  return merge_repeats(path , ratio , transcript , sample_rate )

In [14]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()
labels = bundle.get_labels()
sr = bundle.sample_rate
# get_times(SPEECH_FILE , transcript , bundle , model , labels)

In [21]:
get_times(df.iloc[0]["audio_path"][3:] , df.iloc[0]['text'] , bundle , model , labels , sr)

: 

: 

# RAN IT USING SLURM SO NOW WE WILL SEE THE RESULTS


In [41]:
df = pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data_1.pkl")
df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
199,3,19,18,"i mean , well , 'cause when i was growing up ,...",43,val,../../data/val_splits_wav/dia19_utt18.wav,dia19_utt18,sadness,6,166229,997374,../../data/val_video/dia19_utt18.mp4,negative,0,"(0.12010765895953758, 10.369294556840078)"
200,0,243,3,"okay , very cute braces anyway y know what , the",11,test,../../data/test_splits_wav/dia243_utt3.wav,dia243_utt3,neutral,6,166229,997374,../../data/test_video/dia243_utt3.mp4,neutral,1,"(1.100986873795761, 5.204665221579961)"
201,2,12,6,"well , i i got this blinding pain in my stomac...",34,train,../../data/train_splits_wav/dia12_utt6.wav,dia12_utt6,fear,6,166229,997374,../../data/train_video/dia12_utt6.mp4,negative,0,"(0.02001794315992293, 9.588594773603083)"
202,1,4,11,"ross geller , why do i know that name ? it s u...",20,val,../../data/val_splits_wav/dia4_utt11.wav,dia4_utt11,surprise,6,166229,997374,../../data/val_video/dia4_utt11.mp4,positive,2,"(0.22019737475915221, 6.105472663776493)"
203,3,84,2,"well , i i got this blinding pain in my stomac...",34,test,../../data/test_splits_wav/dia84_utt2.wav,dia84_utt2,sadness,6,166229,997374,../../data/test_video/dia84_utt2.mp4,negative,0,"(0.3002691473988439, 10.229168954720617)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13699,1,516,2,oh i m sorry ! do you need a break ?,11,train,../../data/train_splits_wav/dia516_utt2.wav,dia516_utt2,surprise,2,2048,4096,../../data/train_video/dia516_utt2.mp4,negative,0,None
13700,1,509,14,ow ! ow ! contraction ow ow ! ow ow !,11,train,../../data/train_splits_wav/dia509_utt14.wav,dia509_utt14,surprise,2,2048,4096,../../data/train_video/dia509_utt14.mp4,negative,0,None
13701,4,468,4,i m so glad you could make it,8,train,../../data/train_splits_wav/dia468_utt4.wav,dia468_utt4,joy,2,2048,4096,../../data/train_video/dia468_utt4.mp4,positive,2,None
13702,4,103,6,i love hiking !,4,train,../../data/train_splits_wav/dia103_utt6.wav,dia103_utt6,joy,2,2048,4096,../../data/train_video/dia103_utt6.mp4,positive,2,None


In [68]:
df['timings'][1]

10.369294556840078

In [71]:
not_null = df[~df['timings'].isna()]

In [74]:
not_null[not_null['timings'] < 1]

SyntaxError: invalid syntax (3608836150.py, line 1)

In [75]:
not_null[['start', 'end']] = pd.DataFrame(not_null['timings'].tolist(), index=not_null.index)
not_null

/home/prsood/projects/def-whkchun/prsood/sarcasm_venv/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings,start,end
199,3,19,18,"i mean , well , 'cause when i was growing up ,...",43,val,../../data/val_splits_wav/dia19_utt18.wav,dia19_utt18,sadness,6,166229,997374,../../data/val_video/dia19_utt18.mp4,negative,0,"(0.12010765895953758, 10.369294556840078)",0.120108,10.369295
200,0,243,3,"okay , very cute braces anyway y know what , the",11,test,../../data/test_splits_wav/dia243_utt3.wav,dia243_utt3,neutral,6,166229,997374,../../data/test_video/dia243_utt3.mp4,neutral,1,"(1.100986873795761, 5.204665221579961)",1.100987,5.204665
201,2,12,6,"well , i i got this blinding pain in my stomac...",34,train,../../data/train_splits_wav/dia12_utt6.wav,dia12_utt6,fear,6,166229,997374,../../data/train_video/dia12_utt6.mp4,negative,0,"(0.02001794315992293, 9.588594773603083)",0.020018,9.588595
202,1,4,11,"ross geller , why do i know that name ? it s u...",20,val,../../data/val_splits_wav/dia4_utt11.wav,dia4_utt11,surprise,6,166229,997374,../../data/val_video/dia4_utt11.mp4,positive,2,"(0.22019737475915221, 6.105472663776493)",0.220197,6.105473
203,3,84,2,"well , i i got this blinding pain in my stomac...",34,test,../../data/test_splits_wav/dia84_utt2.wav,dia84_utt2,sadness,6,166229,997374,../../data/test_video/dia84_utt2.mp4,negative,0,"(0.3002691473988439, 10.229168954720617)",0.300269,10.229169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13683,1,351,14,tramp ? !,3,train,../../data/train_splits_wav/dia351_utt14.wav,dia351_utt14,surprise,2,3072,6144,../../data/train_video/dia351_utt14.mp4,negative,0,"(0.08533333333333333, 0.192)",0.085333,0.192000
13685,6,445,13,stay calm ? ! !,5,train,../../data/train_splits_wav/dia445_utt13.wav,dia445_utt13,anger,2,3072,6144,../../data/train_video/dia445_utt13.mp4,negative,0,"(0.0, 0.192)",0.000000,0.192000
13687,0,123,7,he,1,test,../../data/test_splits_wav/dia123_utt7.wav,dia123_utt7,neutral,2,3072,6144,../../data/test_video/dia123_utt7.mp4,neutral,1,"(0.064, 0.10666666666666666)",0.064000,0.106667
13696,0,257,3,oh,1,train,../../data/train_splits_wav/dia257_utt3.wav,dia257_utt3,neutral,2,2048,4096,../../data/train_video/dia257_utt3.mp4,neutral,1,"(0.08533333333333333, 0.128)",0.085333,0.128000


In [89]:
not_null[(not_null['audio_shape'] > 20000)]

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings,start,end
10656,1,173,1,what ? !,3,train,../../data/train_splits_wav/dia173_utt1.wav,dia173_utt1,surprise,6,19797,118782,../../data/train_video/dia173_utt1.mp4,negative,0,"(0.24340573770491808, 0.5679467213114755)",0.243406,0.567947
10657,1,503,2,are you serious ? !,5,train,../../data/train_splits_wav/dia503_utt2.wav,dia503_utt2,surprise,6,19797,118782,../../data/train_video/dia503_utt2.mp4,negative,0,"(0.3245409836065574, 0.9736229508196723)",0.324541,0.973623
10658,0,703,2,cleveland,1,train,../../data/train_splits_wav/dia703_utt2.wav,dia703_utt2,neutral,6,19797,118782,../../data/train_video/dia703_utt2.mp4,neutral,1,"(0.40567622950819676, 1.0547581967213115)",0.405676,1.054758
10659,3,164,4,oh,1,train,../../data/train_splits_wav/dia164_utt4.wav,dia164_utt4,sadness,6,19797,118782,../../data/train_video/dia164_utt4.mp4,negative,0,"(1.0953258196721314, 1.135893442622951)",1.095326,1.135893
10660,6,295,20,yeah !,2,train,../../data/train_splits_wav/dia295_utt20.wav,dia295_utt20,anger,6,19797,118782,../../data/train_video/dia295_utt20.mp4,negative,0,"(0.750501024590164, 0.9736229508196723)",0.750501,0.973623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13683,1,351,14,tramp ? !,3,train,../../data/train_splits_wav/dia351_utt14.wav,dia351_utt14,surprise,2,3072,6144,../../data/train_video/dia351_utt14.mp4,negative,0,"(0.08533333333333333, 0.192)",0.085333,0.192000
13685,6,445,13,stay calm ? ! !,5,train,../../data/train_splits_wav/dia445_utt13.wav,dia445_utt13,anger,2,3072,6144,../../data/train_video/dia445_utt13.mp4,negative,0,"(0.0, 0.192)",0.000000,0.192000
13687,0,123,7,he,1,test,../../data/test_splits_wav/dia123_utt7.wav,dia123_utt7,neutral,2,3072,6144,../../data/test_video/dia123_utt7.mp4,neutral,1,"(0.064, 0.10666666666666666)",0.064000,0.106667
13696,0,257,3,oh,1,train,../../data/train_splits_wav/dia257_utt3.wav,dia257_utt3,neutral,2,2048,4096,../../data/train_video/dia257_utt3.mp4,neutral,1,"(0.08533333333333333, 0.128)",0.085333,0.128000


In [50]:
idx = 1
print(df[df['timings'].isna()].iloc[idx]['text'])
print(df[df['timings'].isna()].iloc[idx]['audio_path'])



but , i was just thinking that since those guys just got engaged that maybe it would be nice if they had some privacy , y know ?
../../data/train_splits_wav/dia505_utt8.wav


In [52]:
null_timing_df = df[df['timings'].isna()]

In [59]:
_505df = df[df["dialog"] == "505"]

In [61]:
_505df[_505df['utterance'] == "7"]['text']

2328    well , umm , not much
Name: text, dtype: object

In [108]:
timing_df = pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data_1.pkl")
orig_df = pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")

In [109]:
len(timing_df),len(orig_df[orig_df["size_padding"] < 999426])

(13505, 13505)

In [102]:
timing_df.columns

Index(['emotion', 'dialog', 'utterance', 'text', 'num_words', 'split',
       'audio_path', 'name', 'emotion_label', 'num_channels', 'audio_shape',
       'size_padding', 'video_path', 'sentiment_label', 'sentiment',
       'timings'],
      dtype='object')

In [110]:
timing_df.drop( ['emotion', 'dialog', 'utterance', 'text', 'num_words', 'split',
       'audio_path', 'name', 'emotion_label', 'num_channels', 'audio_shape',
       'size_padding' , 'sentiment_label', 'sentiment',
       ], axis=1 , inplace = True)

In [111]:
timing_df

,video_path,timings
199,../../data/val_video/dia19_utt18.mp4,"(0.12010765895953758, 10.369294556840078)"
200,../../data/test_video/dia243_utt3.mp4,"(1.100986873795761, 5.204665221579961)"
201,../../data/train_video/dia12_utt6.mp4,"(0.02001794315992293, 9.588594773603083)"
202,../../data/val_video/dia4_utt11.mp4,"(0.22019737475915221, 6.105472663776493)"
203,../../data/test_video/dia84_utt2.mp4,"(0.3002691473988439, 10.229168954720617)"
...,...,...
13699,../../data/train_video/dia516_utt2.mp4,None
13700,../../data/train_video/dia509_utt14.mp4,None
13701,../../data/train_video/dia468_utt4.mp4,None
13702,../../data/train_video/dia103_utt6.mp4,None


In [113]:
new_shorter_df = pd.merge(orig_df , timing_df , on = "video_path")
new_shorter_df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
0,3,19,18,"i mean , well , 'cause when i was growing up ,...",43,val,../../data/val_splits_wav/dia19_utt18.wav,dia19_utt18,sadness,6,166229,997374,../../data/val_video/dia19_utt18.mp4,negative,0,"(0.12010765895953758, 10.369294556840078)"
1,0,243,3,"okay , very cute braces anyway y know what , the",11,test,../../data/test_splits_wav/dia243_utt3.wav,dia243_utt3,neutral,6,166229,997374,../../data/test_video/dia243_utt3.mp4,neutral,1,"(1.100986873795761, 5.204665221579961)"
2,2,12,6,"well , i i got this blinding pain in my stomac...",34,train,../../data/train_splits_wav/dia12_utt6.wav,dia12_utt6,fear,6,166229,997374,../../data/train_video/dia12_utt6.mp4,negative,0,"(0.02001794315992293, 9.588594773603083)"
3,1,4,11,"ross geller , why do i know that name ? it s u...",20,val,../../data/val_splits_wav/dia4_utt11.wav,dia4_utt11,surprise,6,166229,997374,../../data/val_video/dia4_utt11.mp4,positive,2,"(0.22019737475915221, 6.105472663776493)"
4,3,84,2,"well , i i got this blinding pain in my stomac...",34,test,../../data/test_splits_wav/dia84_utt2.wav,dia84_utt2,sadness,6,166229,997374,../../data/test_video/dia84_utt2.mp4,negative,0,"(0.3002691473988439, 10.229168954720617)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13500,1,516,2,oh i m sorry ! do you need a break ?,11,train,../../data/train_splits_wav/dia516_utt2.wav,dia516_utt2,surprise,2,2048,4096,../../data/train_video/dia516_utt2.mp4,negative,0,None
13501,1,509,14,ow ! ow ! contraction ow ow ! ow ow !,11,train,../../data/train_splits_wav/dia509_utt14.wav,dia509_utt14,surprise,2,2048,4096,../../data/train_video/dia509_utt14.mp4,negative,0,None
13502,4,468,4,i m so glad you could make it,8,train,../../data/train_splits_wav/dia468_utt4.wav,dia468_utt4,joy,2,2048,4096,../../data/train_video/dia468_utt4.mp4,positive,2,None
13503,4,103,6,i love hiking !,4,train,../../data/train_splits_wav/dia103_utt6.wav,dia103_utt6,joy,2,2048,4096,../../data/train_video/dia103_utt6.mp4,positive,2,None


In [114]:
new_shorter_df.to_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")

In [115]:
len(pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl"))

13505

In [93]:
orig_df.to_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")